In [ ]:
!pip install pickle5

In [ ]:
import numpy as np
import pandas as pd
import time

import random

import matplotlib
from matplotlib import pyplot as plt

import tensorflow as tf
import tensorflow_probability as tfp
from tqdm import tqdm

import pickle5 as pickle
from scipy import stats

import matplotlib.ticker as tick

import sys
sys.path.append('..')
#from nsgp_vi import nsgpVI

# We'll use double precision throughout for better numerics.
dtype = np.float64

tfb = tfp.bijectors
tfd = tfp.distributions
tfk = tfp.math.psd_kernels

plt.style.use('ggplot') 
plt.style.use('seaborn-paper')
plt.style.use('seaborn-whitegrid')


In [ ]:
# Extracted from https://github.com/OpenNMT/OpenNMT-tf/blob/master/opennmt/optimizers/utils.py

import tensorflow as tf

class GradientAccumulator(object):
    """Gradient accumulation utility.
    When used with a distribution strategy, the accumulator should be called in a
    replica context. Gradients will be accumulated locally on each replica and
    without synchronization. Users should then call ``.gradients``, scale the
    gradients if required, and pass the result to ``apply_gradients``.
    """

    # We use the ON_READ synchronization policy so that no synchronization is
    # performed on assignment. To get the value, we call .value() which returns the
    # value on the current replica without synchronization.

    def __init__(self):
        """Initializes the accumulator."""
        self._gradients = []
        self._accum_steps = None

    @property
    def step(self):
        """Number of accumulated steps."""
        if self._accum_steps is None:
            self._accum_steps = tf.Variable(
                tf.constant(0, dtype=tf.int64),
                trainable=False,
                synchronization=tf.VariableSynchronization.ON_READ,
                aggregation=tf.VariableAggregation.ONLY_FIRST_REPLICA,
            )
        return self._accum_steps.value()

    @property
    def gradients(self):
        """The accumulated gradients on the current replica."""
        if not self._gradients:
            raise ValueError(
                "The accumulator should be called first to initialize the gradients"
            )
        #return list(gradient.value()/tf.cast(self._accum_steps, gradient.dtype) for gradient in self._gradients)
        grads = list(gradient.value()/tf.cast(self._accum_steps, gradient.dtype) for gradient in self._gradients)
        clipped_grads, gn = tf.clip_by_global_norm(grads,clip_norm=1.0)
        return clipped_grads
        
    
    

    def __call__(self, gradients):
        """Accumulates :obj:`gradients` on the current replica."""
        if not self._gradients:
            _ = self.step  # Create the step variable.
            self._gradients.extend(
                [
                    tf.Variable(
                        tf.zeros_like(gradient),
                        trainable=False,
                        synchronization=tf.VariableSynchronization.ON_READ,
                    )
                    for gradient in gradients
                ]
            )
        if len(gradients) != len(self._gradients):
            raise ValueError(
                "Expected %s gradients, but got %d"
                % (len(self._gradients), len(gradients))
            )

        for accum_gradient, gradient in zip(self._gradients, gradients):
            accum_gradient.assign_add(gradient, read_value=False)
        self._accum_steps.assign_add(1)

    def reset(self):
        """Resets the accumulated gradients on the current replica."""
        if not self._gradients:
            return
        self._accum_steps.assign(0)
        for gradient in self._gradients:
            gradient.assign(
                tf.zeros(gradient.shape, dtype=gradient.dtype), read_value=False
            )

In [ ]:
"""
Copyright 2021 Colin Torney
Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at
    http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
"""

import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
from tqdm import tqdm

from tensorflow_probability.python.distributions import kullback_leibler

#from utils.gradient_accumulator import GradientAccumulator

tfb = tfp.bijectors
tfd = tfp.distributions
tfk = tfp.math.psd_kernels

dtype = np.float64
# changed here to 3 instead of 2
NUM_LATENT = 3

class nsgpVI(tf.Module):
                                        
    def __init__(self,kernel_len,kernel_amp,kernel_mean,n_inducing_points,inducing_index_points,dataset,num_training_points, init_observation_noise_variance=1e-2,num_sequential_samples=10,num_parallel_samples=10,jitter=1e-6):
               
        self.jitter=jitter
        
        # define the means for all parameters including the mean
        self.mean_len = tf.Variable([0.0], dtype=tf.float64, name='len_mean', trainable=True)
        self.mean_amp = tf.Variable([0.0], dtype=tf.float64, name='var_mean', trainable=True)
        self.mean_mean = tf.Variable([0.0], dtype=tf.float64, name='mean_mean', trainable=True)

        # inducing points for all parameters including the mean, kept fixed
        self.amp_inducing_index_points = tf.Variable(inducing_index_points,dtype=dtype,name='amp_ind_points',trainable=False) #z's for amplitude
        self.len_inducing_index_points = tf.Variable(inducing_index_points,dtype=dtype,name='len_ind_points',trainable=False) #z's for len
        self.mean_inducing_index_points = tf.Variable(inducing_index_points,dtype=dtype,name='mean_ind_points',trainable=False) #z's for mean

        # prior distributions for all parameters including the mean
        self.kernel_len = kernel_len
        self.kernel_amp = kernel_amp
        self.kernel_mean = kernel_mean

        
        #parameters for variational distribution for len,phi(l_z) and var,phi(sigma_z)
        self.variational_inducing_observations_loc = tf.Variable(np.zeros((NUM_LATENT*n_inducing_points),dtype=dtype),name='ind_loc_post')
        self.variational_inducing_observations_scale = tfp.util.TransformedVariable(np.eye(NUM_LATENT*n_inducing_points, dtype=dtype),tfp.bijectors.FillScaleTriL(diag_shift=np.float64(1e-05)),dtype=tf.float64, name='ind_scale_post', trainable=True)

        
        #approximation to the posterior: phi(l_z)
        self.variational_inducing_observations_posterior = tfd.MultivariateNormalLinearOperator(
                                                                      loc=self.variational_inducing_observations_loc,
                                                                      scale=tf.linalg.LinearOperatorLowerTriangular(self.variational_inducing_observations_scale))

        #p(l_z)
        self.inducing_prior = tfd.MultivariateNormalDiag(loc=tf.zeros((NUM_LATENT*n_inducing_points),dtype=tf.float64),name='ind_prior')
        
        self.vgp_observation_noise_variance = tf.Variable(np.log(np.exp(init_observation_noise_variance)-1),dtype=dtype,name='nv', trainable=False)

        self.num_sequential_samples=num_sequential_samples
        self.num_parallel_samples=num_parallel_samples
        
        self.dataset = dataset
        self.num_training_points=num_training_points
        

    def optimize(self, BATCH_SIZE, SEG_LENGTH, NUM_EPOCHS=100):

        strategy = tf.distribute.MirroredStrategy()
        dist_dataset = strategy.experimental_distribute_dataset(self.dataset)

        initial_learning_rate = 1e-1
        steps_per_epoch = self.num_training_points//(BATCH_SIZE*SEG_LENGTH)
        learning_rate = tf.optimizers.schedules.ExponentialDecay(initial_learning_rate=initial_learning_rate,decay_steps=steps_per_epoch,decay_rate=0.99,staircase=True)
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
        accumulator = GradientAccumulator()

        def train_step(inputs):
            x_train_batch, y_train_batch = inputs
            kl_weight = tf.reduce_sum(tf.ones_like(x_train_batch))/self.num_training_points

            with tf.GradientTape(watch_accessed_variables=True) as tape:
                loss = self.variational_loss(observations=y_train_batch,observation_index_points=x_train_batch,kl_weight=kl_weight) 
            grads = tape.gradient(loss, self.trainable_variables)
            return loss, grads

        @tf.function
        def distributed_train_step(dataset_inputs):
            per_replica_losses, per_replica_grads = strategy.run(train_step, args=(dataset_inputs,))
            return strategy.reduce(tf.distribute.ReduceOp.SUM, per_replica_losses, axis=None), strategy.reduce(tf.distribute.ReduceOp.SUM, per_replica_grads, axis=None)
            #return strategy.reduce(tf.distribute.ReduceOp.SUM, per_replica_losses, axis=None), [strategy.reduce(tf.distribute.ReduceOp.SUM, prg, axis=None) for prg in per_replica_grads]

        pbar = tqdm(range(NUM_EPOCHS))
        loss_history = np.zeros((NUM_EPOCHS))

        for i in pbar:
            batch_count=0    
            epoch_loss = 0.0
            for batch in self.dataset:
                batch_loss = 0.0
                for s in range(self.num_sequential_samples):
                    loss, grads = distributed_train_step(batch)
                    # accumulate the loss and gradient
                    accumulator(grads)
                    batch_loss += loss.numpy()
                grads = accumulator.gradients
                optimizer.apply_gradients(zip(grads, self.trainable_variables))
                accumulator.reset()
                batch_loss/=self.num_sequential_samples
                epoch_loss+=batch_loss
                batch_count+=1
                pbar.set_description("Loss %f, klen %f" % (epoch_loss/batch_count, self.kernel_len.length_scale.numpy()))
            loss_history[i] = epoch_loss/batch_count

        return loss_history


    def variational_loss(self,observations,observation_index_points,kl_weight=1.0):
        
        kl_penalty = self.surrogate_posterior_kl_divergence_prior()
        recon = self.surrogate_posterior_expected_log_likelihood(observations,observation_index_points)
        return (-recon + kl_weight*kl_penalty)

    
    def surrogate_posterior_kl_divergence_prior(self):
        return kullback_leibler.kl_divergence(self.variational_inducing_observations_posterior,self.inducing_prior) 

    
    def surrogate_posterior_expected_log_likelihood(self,observations,observation_index_points):

        #added mean_vals 
        mean_vals,len_vals, amp_vals = self.get_samples(observation_index_points,S=self.num_parallel_samples) 
        K = self.non_stat_matern12(observation_index_points, len_vals, amp_vals) # BxNxN
        K = K + (tf.eye(tf.shape(K)[-1], dtype=tf.float64) * tf.nn.softplus(self.vgp_observation_noise_variance))

        #added a mean, watch the shape here !
        logpdf = tf.reduce_sum(tf.reduce_mean(tfd.MultivariateNormalTriL(loc = mean_vals[...,0],scale_tril = tf.linalg.cholesky(K)).log_prob((observations[...,0])),axis=0))

        return logpdf
    
    def get_samples(self,observation_index_points,S=1):
        mean, var = self.get_conditional(observation_index_points)
        samples = self.sample_conditional(mean, var, S)
    
        # added a mean on this line and the line below
        mean_samples,len_samples,amp_samples = tf.split(samples,NUM_LATENT,axis=2)
        
        return tf.math.softplus(self.mean_mean + mean_samples),tf.math.softplus(self.mean_len + len_samples), tf.math.softplus(self.mean_amp + amp_samples)
    
    def get_conditional(self, observation_index_points):
        
        Xnew = observation_index_points

        # added Z_mean
        Z_amp = self.amp_inducing_index_points 
        Z_len = self.len_inducing_index_points 
        Z_mean = self.mean_inducing_index_points 
        
        # added a kernel_mean
        kernel_amp = self.kernel_amp
        kernel_len = self.kernel_len
        kernel_mean = self.kernel_mean

        f = self.variational_inducing_observations_loc
        q_sqrt = self.variational_inducing_observations_scale
        M = tf.shape(f)[0]
        
        # add Kmm_mean
        Kmm_amp = tf.linalg.LinearOperatorFullMatrix(kernel_amp.matrix(Z_amp,Z_amp) + self.jitter * tf.eye(M//3, dtype=tf.float64),is_positive_definite=True,is_self_adjoint=True)
        Kmm_len = tf.linalg.LinearOperatorFullMatrix(kernel_len.matrix(Z_len,Z_len) + self.jitter * tf.eye(M//3, dtype=tf.float64),is_positive_definite=True,is_self_adjoint=True)        
        Kmm_mean = tf.linalg.LinearOperatorFullMatrix(kernel_mean.matrix(Z_mean,Z_mean) + self.jitter * tf.eye(M//3, dtype=tf.float64),is_positive_definite=True,is_self_adjoint=True)

        # added Kmm_mean
        Kmm = tf.linalg.LinearOperatorBlockDiag([Kmm_mean,Kmm_len,Kmm_amp])

        # added Kmn_mean
        Kmn_amp = tf.linalg.LinearOperatorFullMatrix(kernel_amp.matrix(Z_amp, Xnew),is_positive_definite=True,is_self_adjoint=True)
        Kmn_len = tf.linalg.LinearOperatorFullMatrix(kernel_len.matrix(Z_len, Xnew),is_positive_definite=True,is_self_adjoint=True)        
        Kmn_mean = tf.linalg.LinearOperatorFullMatrix(kernel_mean.matrix(Z_mean, Xnew),is_positive_definite=True,is_self_adjoint=True)

        #added Kmn_mean
        Kmn = tf.linalg.LinearOperatorBlockDiag([Kmn_mean,Kmn_len,Kmn_amp])

        #added Knn_mean
        Knn_amp = tf.linalg.LinearOperatorFullMatrix(kernel_amp.matrix(Xnew, Xnew),is_positive_definite=True,is_self_adjoint=True)
        Knn_len = tf.linalg.LinearOperatorFullMatrix(kernel_len.matrix(Xnew, Xnew),is_positive_definite=True,is_self_adjoint=True)
        Knn_mean = tf.linalg.LinearOperatorFullMatrix(kernel_mean.matrix(Xnew, Xnew),is_positive_definite=True,is_self_adjoint=True)

        #added Knn_mean
        Knn = tf.linalg.LinearOperatorBlockDiag([Knn_mean,Knn_len,Knn_amp])

        mean,var = self.full_conditional_lo(Kmn,Kmm,Knn,f,q_sqrt)
        
        return mean, var

    def get_marginal(self, observation_index_points):
        
        Xnew = observation_index_points

        # added Knn_mean
        Z_amp = self.amp_inducing_index_points 
        Z_len = self.len_inducing_index_points 
        Z_mean = self.mean_inducing_index_points 


        # added kernel_mean
        kernel_amp = self.kernel_amp
        kernel_len = self.kernel_len
        kernel_mean = self.kernel_mean


        f = self.variational_inducing_observations_loc
        q_sqrt = self.variational_inducing_observations_scale


        
        M = tf.shape(f)[0]
        
        #added Kmm_mean
        Kmm_amp = tf.linalg.LinearOperatorFullMatrix(kernel_amp.matrix(Z_amp,Z_amp) + self.jitter * tf.eye(M//3, dtype=tf.float64),is_positive_definite=True,is_self_adjoint=True)
        Kmm_len = tf.linalg.LinearOperatorFullMatrix(kernel_len.matrix(Z_len,Z_len) + self.jitter * tf.eye(M//3, dtype=tf.float64),is_positive_definite=True,is_self_adjoint=True)
        Kmm_mean = tf.linalg.LinearOperatorFullMatrix(kernel_mean.matrix(Z_mean,Z_mean) + self.jitter * tf.eye(M//3, dtype=tf.float64),is_positive_definite=True,is_self_adjoint=True)


        
        Kmm = tf.linalg.LinearOperatorBlockDiag([Kmm_mean,Kmm_len,Kmm_amp])

        # added Kmn_mean
        Kmn_amp = tf.linalg.LinearOperatorFullMatrix(kernel_amp.matrix(Z_amp, Xnew),is_positive_definite=True,is_self_adjoint=True)
        Kmn_len = tf.linalg.LinearOperatorFullMatrix(kernel_len.matrix(Z_len, Xnew),is_positive_definite=True,is_self_adjoint=True)
        Kmn_mean = tf.linalg.LinearOperatorFullMatrix(kernel_mean.matrix(Z_mean, Xnew),is_positive_definite=True,is_self_adjoint=True)


        # added Kmn_mean
        Kmn = tf.linalg.LinearOperatorBlockDiag([Kmn_mean,Kmn_len,Kmn_amp])

        #added Knn_mean
        Knn_amp = tf.linalg.LinearOperatorFullMatrix(kernel_amp.matrix(Xnew, Xnew),is_positive_definite=True,is_self_adjoint=True)
        Knn_len = tf.linalg.LinearOperatorFullMatrix(kernel_len.matrix(Xnew, Xnew),is_positive_definite=True,is_self_adjoint=True)
        Knn_mean = tf.linalg.LinearOperatorFullMatrix(kernel_mean.matrix(Xnew, Xnew),is_positive_definite=True,is_self_adjoint=True)

        Knn = tf.linalg.LinearOperatorBlockDiag([Knn_mean,Knn_len,Knn_amp])

        mean,var = self.marginal(Kmn.to_dense(),Kmm.to_dense(),Knn.to_dense(),f,q_sqrt)
        
        mean_list = tf.split(mean,NUM_LATENT,axis=1)
        var_list = tf.split(var,NUM_LATENT,axis=0)

        return mean_list, var_list
        

    def sample_conditional(self, mean, var, S=1):
        # mean BxNx1
        # var BxNxN
        # returns SxBxNx1
        B = tf.shape(mean)[0]
        N = tf.shape(mean)[1]
        z = tf.random.normal((S,B,N,1),dtype=tf.float64)
        
        I = self.jitter * tf.eye(N, dtype=tf.float64) #NN
        chol = tf.linalg.cholesky(var + I)  # BNN
        samples = mean + tf.matmul(chol, z)#[:, :, :, 0]  # BSN1

        return samples

    def full_conditional(self, Kmn, Kmm, Knn, f, q_sqrt, full_cov=True):

        f = tf.expand_dims(f,-1)
        q_sqrt= tf.expand_dims(q_sqrt,0)

        Lm = tf.linalg.cholesky(Kmm)

        N = tf.shape(Kmn)[-1]
        M = tf.shape(f)[0]

        # Compute the projection matrix A
        Lm = tf.broadcast_to(Lm, tf.shape(Lm))
        A = tf.linalg.triangular_solve(Lm, Kmn, lower=True)  # [..., M, N]

        # compute the covariance due to the conditioning
        fvar = Knn - tf.linalg.matmul(A, A, transpose_a=True)  # [..., N, N]

        # construct the conditional mean
        f_shape = [M, 1]
        f = tf.broadcast_to(f, f_shape)  # [..., M, R]
        fmean = tf.linalg.matmul(A, f, transpose_a=True)  # [..., N, R]

        L = tf.linalg.band_part(q_sqrt, -1, 0)  

        LTA = tf.linalg.matmul(L, A, transpose_a=True)  # [R, M, N]

        fvar = fvar + tf.linalg.matmul(LTA, LTA, transpose_a=True)  # [R, N, N]

        return fmean, fvar
    
    def full_conditional_lo(self, Kmn, Kmm, Knn, f, q_sqrt, full_cov=True):

        f = tf.expand_dims(f,0)
        q_sqrt= tf.expand_dims(q_sqrt,0)

        Lm  = Kmm.cholesky()
        A = Lm.solve(Kmn)
        
        fmean = A.matvec(f,adjoint=True)

        B = A.matmul(A,adjoint=True)


        L = tf.linalg.LinearOperatorLowerTriangular(q_sqrt)

        LTA = L.matmul(A,adjoint=True)

        LTA = LTA.matmul(LTA,adjoint=True)

        fvar = Knn.to_dense() - B.to_dense() + LTA.to_dense()
        
        return tf.expand_dims(fmean,-1), fvar

    def marginal(self, Kmn, Kmm, Knn, f, q_sqrt, full_cov=True):

        f = tf.expand_dims(f,-1)
        q_sqrt= tf.expand_dims(q_sqrt,0)

        Knn = tf.linalg.diag_part(Knn)
        Lm = tf.linalg.cholesky(Kmm)

        N = tf.shape(Kmn)[-1]
        M = tf.shape(f)[0]

        # Compute the projection matrix A
        Lm = tf.broadcast_to(Lm, tf.shape(Lm))
        A = tf.linalg.triangular_solve(Lm, Kmn, lower=True)  # [..., M, N]

        # compute the covariance due to the conditioning
        fvar = Knn - tf.reduce_sum(tf.square(A), -2)  # [..., N]

        # construct the conditional mean
        f_shape = [M, 1]
        f = tf.broadcast_to(f, f_shape)  # [..., M, R]
        fmean = tf.linalg.matmul(A, f, transpose_a=True)  # [..., N, R]

        L = tf.linalg.band_part(q_sqrt, -1, 0)  

        LTA = tf.linalg.matmul(L, A, transpose_a=True)  # [R, M, N]

        fvar = fvar + tf.reduce_sum(tf.square(LTA), -2)  # [R, N]
        fvar = tf.linalg.adjoint(fvar)  # [N, R]

        return fmean, fvar
    
        
    def non_stat_matern12(self, X, lengthscales, stddev):
        ''' Non-stationary Matern 12 kernel'''
        
        Xs = tf.reduce_sum(input_tensor=tf.square(X), axis=-1, keepdims=True)#(1000,1)
        Ls = tf.square(lengthscales)#(1,1000,1)
        
        dist = -2 * tf.matmul(X, X, transpose_b=True)
        dist += Xs + tf.linalg.matrix_transpose(Xs)
        Lscale = Ls + tf.linalg.matrix_transpose(Ls)
        dist = tf.divide(2*dist,Lscale)
        dist = tf.sqrt(tf.maximum(dist, 1e-40))
        prefactL = 2 * tf.matmul(lengthscales, lengthscales, transpose_b=True)
        prefactV = tf.matmul(stddev, stddev, transpose_b=True)

        return tf.multiply(prefactV,tf.multiply( tf.sqrt(tf.maximum(tf.divide(prefactL,Lscale), 1e-40)),tf.exp(-dist)))


    def non_stat_vel(self,T,lengthscales, stddev):
        
        """Non-stationary integrated Matern12 kernel"""

        sigma_ = 0.5*(stddev[...,:-1,0,None] + stddev[...,1:,0,None])
        len_ = 0.5*(lengthscales[...,:-1,0,None] + lengthscales[...,1:,0,None])

        Ls = tf.square(len_)

        L = tf.math.sqrt(0.5*(Ls + tf.linalg.matrix_transpose(Ls)))

        prefactL = tf.math.sqrt(tf.matmul(len_, len_, transpose_b=True))
        prefactV = tf.matmul(sigma_, sigma_,transpose_b=True)

        zeta = tf.math.multiply(prefactV,tf.math.divide(prefactL,L))
    

        tpq1 = tf.math.exp(tf.math.divide(-tf.math.abs(tf.linalg.matrix_transpose(T[:-1]) - T[1:]),L))
        tp1q1 = tf.math.exp(tf.math.divide(-tf.math.abs(tf.linalg.matrix_transpose(T[1:]) - T[1:]),L))
        tpq = tf.math.exp(tf.math.divide(-tf.math.abs(tf.linalg.matrix_transpose(T[:-1]) - T[:-1]),L))
        tp1q = tf.math.exp(tf.math.divide(-tf.math.abs(tf.linalg.matrix_transpose(T[1:]) - T[:-1]),L))


        Epq_grid = tpq1-tp1q1-tpq+tp1q
        Epq_grid = (L**2)*Epq_grid
                
        Epq_grid = tf.linalg.set_diag(Epq_grid,(tf.linalg.diag_part(Epq_grid)) + 2.0*tf.squeeze(len_)[:]*(tf.squeeze(T[1:])-tf.squeeze(T[:-1])))
        Epq_grid = zeta*Epq_grid
        
        
        K = tf.math.cumsum(tf.math.cumsum(Epq_grid,axis=-2,exclusive=False),axis=-1,exclusive=False)
        
        return K

In [ ]:
print(tfp.__version__)
print(tf.__version__)

In [ ]:
df = pd.read_csv('../input/realvidata/Tetuan City power consumption.csv', thousands=',')

dts = pd.DatetimeIndex(df['DateTime'],dayfirst=True)
T = (24*(dts.day-1) + dts.hour  +dts.minute/60.).astype(float).values[:,None]
# in hours !!!
X = df['Zone 1 Power Consumption'].astype('float').values.reshape(len(T),1)
# 3 zones

X = np.log(X)
#meanX = np.mean(X)
#X = X - meanX
#X = X/X.std()

print(T.shape)
print(X.shape)


In [ ]:
df

In [ ]:
num_training_points_ = T.shape[0]

num_inducing_points_ = 48

inducing_index_points = np.linspace(0., 24., num_inducing_points_, endpoint=False)[..., np.newaxis]
np.random.shuffle(inducing_index_points)


In [ ]:
BATCH_SIZE=8
SEG_LENGTH=500 #1024


class segment_generator:
    def __iter__(self):
        
        # loop over segments
        self.j = 0
        self.max_j = num_training_points_//SEG_LENGTH
        
        
        return self

    def __next__(self):
        

        if self.j==self.max_j:
            raise StopIteration

        TT = T[self.j*SEG_LENGTH:(self.j+1)*SEG_LENGTH]
        XX = X[self.j*SEG_LENGTH:(self.j+1)*SEG_LENGTH]
    
        self.j += 1

        return TT,XX

        
dataset = tf.data.Dataset.from_generator(segment_generator, (tf.float64)) 
dataset = dataset.map(lambda dd: (dd[0],dd[1]))
dataset = dataset.shuffle(1000)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
dataset = tf.data.Dataset.from_generator(segment_generator, (tf.float64)) 
dataset = dataset.map(lambda dd: (dd[0],dd[1]))
dataset = dataset.shuffle(1000)
dataset = dataset.batch(BATCH_SIZE)

In [ ]:
for d in dataset:
    print(d[0].shape,d[1].shape)

In [ ]:
#lengthscale kernel parameters,lower levels
kernel_len_a = tfp.util.TransformedVariable(1.0, tfb.Softplus(),dtype=tf.float64, name='k_len_a',trainable=True)
kernel_len_l = tfp.util.TransformedVariable(1.0, tfb.Chain([tfb.Shift(np.float64(0.5)), tfb.Softplus()]),dtype=tf.float64, name='k_len_l',trainable=True)

# amplitude kernel parameters, lower levels
kernel_amp_a = tfp.util.TransformedVariable(1.0, tfb.Softplus(),dtype=tf.float64, name='k_amp_a',trainable=True)
kernel_amp_l = tfp.util.TransformedVariable(1.0, tfb.Chain([tfb.Shift(np.float64(0.5)), tfb.Softplus()]),dtype=tf.float64, name='k_amp_l',trainable=True)

# mean kernel parameters, lower levels
kernel_mean_a = tfp.util.TransformedVariable(1.0, tfb.Softplus(),dtype=tf.float64, name='k_mean_a',trainable=True)
kernel_mean_l = tfp.util.TransformedVariable(1.0, tfb.Chain([tfb.Shift(np.float64(0.5)), tfb.Softplus()]),dtype=tf.float64, name='k_mean_l',trainable=True)

#kernels on the second layer
kernel_len = tfk.ExpSinSquared(kernel_len_a,kernel_len_l,period=np.float64(24.0))
kernel_amp = tfk.ExpSinSquared(kernel_amp_a,kernel_amp_l,period=np.float64(24.0))
kernel_mean = tfk.ExpSinSquared(kernel_mean_a,kernel_mean_l,period=np.float64(24.0))

#added kernel_mean
vgp = nsgpVI(kernel_len,kernel_amp,kernel_mean,n_inducing_points=num_inducing_points_,inducing_index_points=inducing_index_points,dataset=dataset,num_training_points=num_training_points_, num_sequential_samples=20,num_parallel_samples=10,init_observation_noise_variance=0.005**2)  


In [ ]:
#vgp.trainable_variables

In [ ]:
loss = vgp.optimize(BATCH_SIZE, SEG_LENGTH, NUM_EPOCHS=500)


In [ ]:
#ZZ = np.linspace(0,24*60,200)[:,None]
ZZ = np.linspace(0,24,200)[:,None]

[mean_mean,len_mean,amp_mean], [mean_var,len_var,amp_var] = vgp.get_marginal(ZZ[None,...])

mean_mean = mean_mean[0,:,0].numpy()
mean_std = mean_var[:,0].numpy()**0.5


len_mean = len_mean[0,:,0].numpy()
len_std = len_var[:,0].numpy()**0.5

amp_mean = amp_mean[0,:,0].numpy()
amp_std = amp_var[:,0].numpy()**0.5


f, (ax1, ax2,ax3) = plt.subplots(1, 3,figsize=(30,10))

ax3.plot(ZZ,tf.math.softplus(vgp.mean_mean + mean_mean),color='C1')
ax3.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_mean + mean_mean - 1.28*mean_std),tf.math.softplus(vgp.mean_mean + mean_mean + 1.28*mean_std),color='C1',alpha=0.25)
ax3.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_mean + mean_mean - 1.96*mean_std),tf.math.softplus(vgp.mean_mean + mean_mean + 1.96*mean_std),color='C1',alpha=0.25)
ax3.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_mean + mean_mean - 2.58*mean_std),tf.math.softplus(vgp.mean_mean + mean_mean + 2.58*mean_std),color='C1',alpha=0.25)

ax1.plot(ZZ,tf.math.softplus(vgp.mean_len + len_mean),color='C1')
ax1.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_len + len_mean - 1.28*len_std),tf.math.softplus(vgp.mean_len + len_mean + 1.28*len_std),color='C1',alpha=0.25)
ax1.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_len + len_mean - 1.96*len_std),tf.math.softplus(vgp.mean_len + len_mean + 1.96*len_std),color='C1',alpha=0.25)
ax1.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_len + len_mean - 2.58*len_std),tf.math.softplus(vgp.mean_len + len_mean + 2.58*len_std),color='C1',alpha=0.25)

ax2.plot(ZZ,tf.math.softplus(vgp.mean_amp + amp_mean),color='C1')
ax2.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_amp + amp_mean - 1.28*amp_std),tf.math.softplus(vgp.mean_amp + amp_mean + 1.28*amp_std),color='C1',alpha=0.25)
ax2.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_amp + amp_mean - 1.96*amp_std),tf.math.softplus(vgp.mean_amp + amp_mean + 1.96*amp_std),color='C1',alpha=0.25)
ax2.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_amp + amp_mean - 2.58*amp_std),tf.math.softplus(vgp.mean_amp + amp_mean + 2.58*amp_std),color='C1',alpha=0.25)

plt.show()


In [ ]:
#!pip install pickle5

In [ ]:
import pickle5 as pickle

outputvars = []

for v in vgp.trainable_variables:
    outputvars.append(v.numpy())
    
#make sure you change the names for each replicate dataset !       
with open('opt_power_1mil.pkl', 'wb') as f:
    pickle.dump(outputvars, f)


In [ ]:
np.save('T_ind_power_1mil',inducing_index_points)    

In [ ]:
import pickle5 as pickle

#Load the inducing points and the optimized parameters 
with open('../input/powerresultsvi/power.pkl', 'rb') as f:
    loadp = pickle.load(f)
    
inducing_index_points = np.load('../input/powerresultsvi/T_ind_power.npy')    

In [ ]:
vgp = nsgpVI(kernel_len,kernel_amp,n_inducing_points=num_inducing_points_,inducing_index_points=inducing_index_points,dataset=dataset,num_training_points=num_training_points_, num_sequential_samples=20,num_parallel_samples=10,init_observation_noise_variance=0.005**2)  


In [ ]:
#Load the parameters !!!
for np_v, tf_v in zip(loadp,vgp.trainable_variables):
    tf_v.assign(np_v)

In [ ]:

ZZ = np.linspace(0,24,200)[:,None]

[len_mean,amp_mean], [len_var,amp_var] = vgp.get_marginal(ZZ[None,...])


# len_mean, len_var = vgp.get_len_cond(ZZ[None,...],full_cov=False)
len_mean = len_mean[0,:,0].numpy()
len_std = len_var[:,0].numpy()**0.5



# amp_mean, amp_var = vgp.get_amp_cond(ZZ[None,...],full_cov=False)
amp_mean = amp_mean[0,:,0].numpy()
amp_std = amp_var[:,0].numpy()**0.5


f, (ax1, ax2) = plt.subplots(1, 2,figsize=(16,6))


ax1.plot(ZZ,tf.math.softplus(vgp.mean_len + len_mean),color='C1')
ax1.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_len + len_mean - 1.28*len_std),tf.math.softplus(vgp.mean_len + len_mean + 1.28*len_std),color='C1',alpha=0.25)
ax1.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_len + len_mean - 1.96*len_std),tf.math.softplus(vgp.mean_len + len_mean + 1.96*len_std),color='C1',alpha=0.25)
ax1.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_len + len_mean - 2.58*len_std),tf.math.softplus(vgp.mean_len + len_mean + 2.58*len_std),color='C1',alpha=0.25)

ax1.set_xlabel('Time',size=15)
ax1.set_ylabel('Average power consumption persistence',size=15)
ax1.text(-0.05,1,'A', size=20, transform=ax1.transAxes)
ax1.tick_params(axis='both', which='major', labelsize=15)
tick_locator = tick.MaxNLocator(nbins=7)

ax2.plot(ZZ,tf.math.softplus(vgp.mean_amp + amp_mean),color='C1')
ax2.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_amp + amp_mean - 1.28*amp_std),tf.math.softplus(vgp.mean_amp + amp_mean + 1.28*amp_std),color='C1',alpha=0.25)
ax2.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_amp + amp_mean - 1.96*amp_std),tf.math.softplus(vgp.mean_amp + amp_mean + 1.96*amp_std),color='C1',alpha=0.25)
ax2.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_amp + amp_mean - 2.58*amp_std),tf.math.softplus(vgp.mean_amp + amp_mean + 2.58*amp_std),color='C1',alpha=0.25)

ax2.set_xlabel('Time',size=15)
ax2.set_ylabel('Average power consumption variance',size=15)
ax2.text(-0.1,1,'B', size=20, transform=ax2.transAxes)
ax2.tick_params(axis='both', which='major', labelsize=15)
tick_locator = tick.MaxNLocator(nbins=7)

plt.savefig("power_inference_vi.pdf",dpi=600)

plt.show()
